API

webpage: https://platform.fatsecret.com/

In [2]:
from requests_oauthlib import OAuth2Session
import requests
import pandas as pd
import pandas as pd
import time

obtain token: 

In [ ]:
client_id = "3f4989514a1a4e6db903d491f35cb259"
client_secret = "0c094b0cded14be7a480e392b9746344"

token_url = 'https://oauth.fatsecret.com/connect/token'

data = {'grant_type': 'client_credentials', 
        'scope': 'basic'  
        }

response = requests.post(token_url, data=data, auth=(client_id, client_secret))
token = response.json().get('access_token')

print("Access Token:", token)


extract API

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/x-www-form-urlencoded"
    }

all_foods = []
search_term = "apple"
page = 1
max_results = 20

while True:
    data = {
        "method": "foods.search",
        "search_expression": search_term,
        "format": "json",
        "page_number": page,
        "max_results": max_results
        }

    response = requests.post("https://platform.fatsecret.com/rest/server.api", headers=headers, data=data)
    result = response.json()

    foods = result['foods']['food']
    all_foods.extend(foods)

    total_results = int(result['foods']['total_results'])
    if (page + 1) * max_results >= total_results:
        break

    page += 1
    time.sleep(0.5) 


Turn Data into pandas DF and save 

In [ ]:
data = response.json()

foods = data['foods']['food']

data_list = []

for item in foods:
    food_data = {
        'food_name': item['food_name'],
        'brand_name': item.get('brand_name', 'Generic'),
        'description': item['food_description'],
        'type': item['food_type'],
        'url': item['food_url']
    }
    data_list.append(food_data)

df = pd.DataFrame(data_list)

df.to_csv("foods_data.csv", index=False)

print(df.head())

In [40]:
pd.read_csv('recipes_api.csv')

,food_name,brand_name,description,type,url
0,Red Delicious Apple (Large),Publix,Per 1 apple - Calories: 110kcal | Fat: 0.00g |...,Brand,https://www.fatsecret.com/calories-nutrition/p...
1,Fuji Apples,Archer Farms,Per 1 medium - Calories: 80kcal | Fat: 0.00g |...,Brand,https://www.fatsecret.com/calories-nutrition/a...
2,Apple Slices (Cup),Fresh & Easy,Per 1 cup - Calories: 80kcal | Fat: 0.00g | Ca...,Brand,https://www.fatsecret.com/calories-nutrition/f...
3,Apple Cider Flavored Drink (made From Powdered...,Generic,Per 228g - Calories: 82kcal | Fat: 0.00g | Car...,Generic,https://www.fatsecret.com/calories-nutrition/g...
4,Braeburn Apples,Generic,Per 99g - Calories: 51kcal | Fat: 0.17g | Carb...,Generic,https://www.fatsecret.com/calories-nutrition/g...
...,...,...,...,...,...
1998,Oven Roasted Chicken Breast,Applegate Farms,Per 3 slices - Calories: 50kcal | Fat: 1.50g |...,Brand,https://www.fatsecret.com/calories-nutrition/a...
1999,Smoked Chicken Breast,Applegate Farms,Per 2 oz - Calories: 70kcal | Fat: 2.00g | Car...,Brand,https://www.fatsecret.com/calories-nutrition/a...
2000,Herb Turkey Breast,Applegate Farms,Per 2 oz - Calories: 50kcal | Fat: 0.50g | Car...,Brand,https://www.fatsecret.com/calories-nutrition/a...
2001,Smoked Turkey Breast,Applegate Farms,Per 2 slices - Calories: 50kcal | Fat: 0.50g |...,Brand,https://www.fatsecret.com/calories-nutrition/a...


Scraping 

https://diabetesfoodhub.org/

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import random

Obtaining all recipe links 

In [ ]:
categories = ['https://diabetesfoodhub.org/recipes/breakfast-and-brunch',
             'https://diabetesfoodhub.org/recipes/lunch',
             'https://diabetesfoodhub.org/recipes/dinner',
              'https://diabetesfoodhub.org/recipes/snacks',
              'https://diabetesfoodhub.org/recipes/dessert',
              'https://diabetesfoodhub.org/recipes/kidney-friendly',
              'https://diabetesfoodhub.org/recipes/vegetarian',
              'https://diabetesfoodhub.org/recipes/low-carb',
              'https://diabetesfoodhub.org/recipes/veggie-rich',
              'https://diabetesfoodhub.org/recipes/budget-friendly',
              'https://diabetesfoodhub.org/recipes/quick-easy',
              'https://diabetesfoodhub.org/recipes/kid-friendly']

In [ ]:

all_recipes = []

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

for category in categories:
    print(f"\nScraping category: {category}")
    base_url = category
    page = 0
    seen_links = set()

    while True:
        url = base_url if page == 0 else f"{base_url}?page={page}"
        resp = session.get(url)
        if resp.status_code == 404:
            print(f"Page not found: {url}")
            break
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        recipe_links = soup.find_all('a', string="View Recipe")
        found_on_this_page = 0

        for a in recipe_links:
            href = a.get("href")
            if not href or href in seen_links:
                continue
            seen_links.add(href)
            title_tag = a.find_previous("h3")
            title = title_tag.get_text(strip=True) if title_tag else "(No title)"
            full_link = href if href.startswith("http") else "https://www.diabetesfoodhub.org" + href

            all_recipes.append({
                "category": category,
                "title": title,
                "link": full_link
            })
            found_on_this_page += 1

        print(f"Page {page}: Found {found_on_this_page} recipes")

        load_more_btn = soup.find(lambda tag: tag.name == "a" and "Load more" in tag.text)
        if not load_more_btn:
            break
        page += 1


print(f"\nTotal recipes collected: {len(all_recipes)}")

In [ ]:
all_recipes_links = [item['link'] for item in all_recipes]

Obtaining the data from the Links

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

def extract_recipe_data(url):
    try:
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        scripts = soup.find_all("script", type="application/ld+json")
        for tag in scripts:
            try:
                data = json.loads(tag.string)
                recipe = None

                if "@graph" in data:
                    for item in data["@graph"]:
                        if item.get("@type") == "Recipe":
                            recipe = item
                            break
                elif data.get("@type") == "Recipe":
                    recipe = data

                if recipe:
                    return {"title": recipe.get("name", ""),
                            "url": url,"ingredients": "; ".join(recipe.get("recipeIngredient", [])),
                            **recipe.get("nutrition", {})}
            except Exception:
                continue
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

    return None

In [ ]:
results = []

for i, link in enumerate(all_recipe_links):
    print(f"({i+1}/{len(all_recipe_links)}) Scraping: {link}")
    data = extract_recipe_data(link)
    if data:
        results.append(data)
    time.sleep(random.uniform(0.5, 1.1))
df = pd.DataFrame(results)

df.to_csv("diabetes_recipes.csv", index=False)

In [22]:
data = pd.read_csv('diabetes_recipes.csv')

In [28]:
data.head()

,title,url,ingredients,@type,servingSize,calories,carbohydrateContent,cholesterolContent,fiberContent,proteinContent,sodiumContent,sugarContent,fatContent,saturatedFatContent,transFatContent
0,Good Morning Power Parfait,https://diabetesfoodhub.org/recipes/good-morni...,Plain Nonfat Greek yogurt\n \n ...,NutritionInformation,"['1 parfait', 'about 1 cup']",150,24g,5mg,4g,14g,80mg,0g,1g,0g,0g
1,Asparagus and Bok Choy Frittata,https://diabetesfoodhub.org/recipes/asparagus-...,olive oil\n \n 2...,NutritionInformation,1 slice,130,6g,5mg,2g,14g,450mg,0g,6g,2g,0g
2,Guilt-Free Breakfast Sausage Patties,https://diabetesfoodhub.org/recipes/guilt-free...,lean ground turkey\n \n ...,NutritionInformation,1 patties=0.5\r2 patties=1\r3 patties=1.5\r4 p...,70,2g,30mg,0g,15g,115mg,g,0.5g,0g,0g
3,Pumpkin-Banana Muffins,https://diabetesfoodhub.org/recipes/pumpkin-ba...,whole wheat flour\n \n ...,NutritionInformation,1 muffin,140,20g,25mg,3g,3g,220mg,5g,6g,0.5g,0g
4,Strawberry Stuffed French Toast,https://diabetesfoodhub.org/recipes/strawberry...,Splenda® Granulated Sweetener\n \n ...,NutritionInformation,½ sandwich,120,15g,55mg,8g,10g,240mg,0g,6g,1.5g,0g


Extra Datasets just two example but theres 7 dfs

In [32]:
pd.read_csv('keto.csv')

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
0,keto,Keto Fat Bombs,american,49.19,79.78,264.40,2022-10-16,17:58:03
1,keto,Keto Egg Bites Recipe,american,92.86,16.19,86.51,2022-10-16,17:58:03
2,keto,Keto Key Lime Mousse,american,27.49,23.58,177.43,2022-10-16,17:58:03
3,keto,Keto Peanut Butter Squares,american,130.00,507.48,362.36,2022-10-16,17:58:03
4,keto,"YOGURT RICE, REINVENTED THE KETO WAY!",mediterranean,21.23,34.78,27.74,2022-10-16,17:58:03
...,...,...,...,...,...,...,...,...
1507,keto,Macadamia Fat Bread,french,53.77,79.92,266.07,2022-10-16,18:09:25
1508,keto,"Beef, cream cheese & shredded cheese",american,169.13,7.26,239.23,2022-10-16,18:09:25
1509,keto,No Carb (No Cauliflower) Pizza Crust,italian,225.20,10.55,94.58,2022-10-16,18:09:25
1510,keto,Indian Coconut Curry Salmon,nordic,62.59,39.71,65.81,2022-10-16,18:09:25


In [34]:
pd.read_csv('paleo.csv')

,Diet_type,Recipe_name,Cuisine_type,Protein(g),Carbs(g),Fat(g),Extraction_day,Extraction_time
0,paleo,Bone Broth From 'Nom Nom Paleo',american,5.22,1.29,3.20,2022-10-16,17:20:09
1,paleo,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",south east asian,181.55,28.62,146.14,2022-10-16,17:20:09
2,paleo,Paleo Pumpkin Pie,american,30.91,302.59,96.76,2022-10-16,17:20:09
3,paleo,Strawberry Guacamole recipes,mexican,9.62,75.78,59.89,2022-10-16,17:20:09
4,paleo,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",chinese,39.84,54.08,71.55,2022-10-16,17:20:09
...,...,...,...,...,...,...,...,...
1269,paleo,Sunday Slow Cooker: Thai Curry Ground Beef rec...,south east asian,104.82,34.94,32.89,2022-10-16,17:31:24
1270,paleo,Paleo Chicken Nuggets recipes,american,211.64,47.69,100.54,2022-10-16,17:31:24
1271,paleo,Paleo Bacon-Wrapped Cauliflower,american,70.23,224.84,424.27,2022-10-16,17:31:24
1272,paleo,Green Eggs and Bacon,american,28.37,13.96,101.73,2022-10-16,17:31:24
